In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras import layers
from keras import models
from keras import optimizers
import tensorflow as tf
import tensorflow.keras.layers as layers
import cv2
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''




# 学習モデルの定義

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Flatten())
 
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=["acc"])
 
model.summary()

# 訓練データの前処理

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#データの正規化 
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
#訓練データ
train_dir = ('../input/use-data/data/train') 
#教師データ
validation_dir = ('../input/use-data/data/vali') 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode="binary"
)
 
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode="binary"
)

# 学習と学習モデルの保存

In [ ]:
#学習開始
history = model.fit_generator(train_generator,
                             steps_per_epoch=6000/32,
                             epochs=30,
                             validation_data=validation_generator,
                             validation_steps=1250/32)
#学習モデルの保存
model.save('../working/dog_cat.h5')


# モデルでの予測

In [ ]:
from keras.preprocessing import image
import re
import csv
import glob

In [ ]:
p = re.compile(r'\d+')
imcount = 1
test_list = glob.glob('../input/use-data/data/test/*.jpg')
test_list = sorted(test_list,key=lambda s: int(p.search(s).group()))

#モデルの読み込み
model = models.load_model('../working/dog_cat.h5')

#csv書き込み
with open('../working/submission.csv', 'w') as f:
  writer= csv.writer(f, lineterminator='\n')
  writer.writerow(['id', 'label'])
  for i in test_list:
    img = image.load_img(i, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    result_predict = model.predict(x)
    writer.writerow([imcount, result_predict[0][0]])
    imcount += 1


